<a href="https://colab.research.google.com/github/divyansh1195/Kaggle-ML-Exercise-Notebooks/blob/master/Pandas_Wines_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I have implemented 'Mapping', 'Grouping', 'Sorting', 'How to take care of Missing Values', 'Renaming and Combining' operations on Wines dataset from Kaggle.

In [ ]:
import pandas as pd
reviews=pd.read_csv('wines.csv', index_col=0)
reviews.head(4)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian


In [ ]:
#Q1. Create a variable df containing the country, province, region_1, and region_2 columns of the records with the index labels 0, 1, 10, and 100.
df_new=reviews.loc[[0,1,10,100],['country', 'province', 'region_1', 'region_2']]
df_new
#whenever there is label indexing use .loc instaed of .iloc

,country,province,region_1,region_2
0,Italy,Sicily & Sardinia,Etna,NaN
1,Portugal,Douro,NaN,NaN
10,US,California,Napa Valley,Napa
100,US,New York,Finger Lakes,Finger Lakes


In [ ]:
#Q2.Create a variable df containing the country and variety columns of the first 100 records.
country_variety_100=df = reviews.loc[0:99,['country','variety']]
country_variety_100

,country,variety
0,Italy,White Blend
1,Portugal,Portuguese Red
2,US,Pinot Gris
3,US,Riesling
4,US,Pinot Noir
...,...,...
95,France,Gamay
96,France,Gamay
97,US,Riesling
98,Italy,Sangiovese


In [ ]:
#Q3.Create a DataFrame italian_wines containing reviews of wines made in Italy
country_italy=reviews[reviews.country=='Italy']
country_italy.head(5)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
13,Italy,This is dominated by oak and oak-driven aromas...,Rosso,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte
22,Italy,Delicate aromas recall white flower and citrus...,Ficiligno,87,19.0,Sicily & Sardinia,Sicilia,NaN,Kerin O’Keefe,@kerinokeefe,Baglio di Pianetto 2007 Ficiligno White (Sicilia),White Blend,Baglio di Pianetto
24,Italy,"Aromas of prune, blackcurrant, toast and oak c...",Aynat,87,35.0,Sicily & Sardinia,Sicilia,NaN,Kerin O’Keefe,@kerinokeefe,Canicattì 2009 Aynat Nero d'Avola (Sicilia),Nero d'Avola,Canicattì


In [ ]:
#Q4.Create a DataFrame top_oceania_wines containing all reviews with at least 95 points (out of 100) for wines from Australia or New Zealand.
oceania_wines=reviews[(reviews.country.isin(['Australia', 'New Zealand'])) & (reviews.points>=95)]
oceania_wines.head(2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
345,Australia,This wine contains some material over 100 year...,Rare,100,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards
346,Australia,"This deep brown wine smells like a damp, mossy...",Rare,98,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscadelle...,Muscadelle,Chambers Rosewood Vineyards


**Exercise: Summary Functions and Maps**

In [ ]:
#Q1.median of the points column in the reviews DataFrame
#median of the points column in the reviews DataFrame
median_points = reviews.points.median()
median_points

88.0

In [ ]:
#Q2.What countries are represented in the dataset? no duplicates
countries = reviews.country.unique()
print(countries)
#Q3.How often does each country appear in the dataset? 
country_counts=reviews.country.value_counts().sort_values(ascending=False)
print(country_counts)#frequency of countries

['Italy' 'Portugal' 'US' 'Spain' 'France' 'Germany' 'Argentina' 'Chile'
 'Australia' 'Austria' 'South Africa' 'New Zealand' 'Israel' 'Hungary'
 'Greece' 'Romania' 'Mexico' 'Canada' nan 'Turkey' 'Czech Republic'
 'Slovenia' 'Luxembourg' 'Croatia' 'Georgia' 'Uruguay' 'England' 'Lebanon'
 'Serbia' 'Brazil' 'Moldova' 'Morocco' 'Peru' 'India' 'Bulgaria' 'Cyprus'
 'Armenia' 'Switzerland' 'Bosnia and Herzegovina' 'Ukraine' 'Slovakia'
 'Macedonia' 'China' 'Egypt']
US                        54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                   

In [ ]:
#Q4.Create variable centered_price containing a version of the price column with the mean price subtracted.
centered_price = reviews.price-reviews.price.mean()
print(centered_price)

0               NaN
1        -20.363389
2        -21.363389
3        -22.363389
4         29.636611
            ...    
129966    -7.363389
129967    39.636611
129968    -5.363389
129969    -3.363389
129970   -14.363389
Name: price, Length: 129971, dtype: float64


In [ ]:
#Q5.
#I'm an economical wine buyer. Which wine is the "best bargain"? 
#Create a variable bargain_wine with the title of the wine with the highest points-to-price ratio in the dataset.
#idxmax() function to find the index of the maximum value along the column axis

#difference between .loc and .iloc
#loc gets rows (or columns) with particular labels from the index.
#iloc gets rows (or columns) at particular positions in the index (so it only takes integers)

points_to_price_ratio=(reviews.points/reviews.price)
#add a column top the dataset in python
reviews['points_to_price_ratio']=points_to_price_ratio
#idxmax returns the label with maximum value
bargain_wine = reviews.loc[points_to_price_ratio.idxmax(), 'title']
bargain_wine

'Bandit NV Merlot (California)'

In [ ]:
#Q6. Is a wine more likely to be "tropical" or "fruity"? 
#Create a Series descriptor_counts counting how many times each of these two words appears in the description column in the dataset.

#Map in Python-
#Python map() function is used to apply a function on all the elements of specified iterable and return map object. 
#Python map object is an iterator, so we can iterate over its elements.

desc_tropical=reviews.description.map(lambda desc: "tropical" in desc).sum()
desc_fruity=reviews.description.map(lambda desc: "fruity" in desc).sum()
descriptor_counts = pd.Series([desc_tropical, desc_fruity], index=['tropical', 'fruity'], name='Descriptor Counts')
descriptor_counts

tropical    3607
fruity      9090
Name: Descriptor Counts, dtype: int64

In [ ]:
#Q7.Create a series star_ratings with the number of stars corresponding to each review in the dataset.
#apply() is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.
def star_points(row):
  """A score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star.
  Also, any wines from Canada should automatically get 3 stars, regardless of points."""
  if row.country=='Canada':
      return 3
  elif row.points >=95:
      return 3 
  elif row.points >=85:
      return 2
  else:
      return 1
star_ratings = reviews.apply(star_points, axis='columns')#adds the star_ratings to the column section 
reviews["star_ratings"]=star_ratings
reviews.head(2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,points_to_price_ratio,star_ratings
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,NaN,2
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,5.8,2


**Exercise: Grouping and Sorting**

In [ ]:
#Q1.most common wine reviewers in the dataset
#Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote
#reviews_written =reviews.groupby('taster_twitter_handle').taster_twitter_handle.count() 
#or,
#reviews_written =reviews.groupby('taster_twitter_handle').size()
#or,
reviews_written =reviews.taster_twitter_handle.value_counts().sort_values(ascending=True)
reviews_written

@winewchristina         6
@bkfiona               27
@worldwineguys       1005
@suskostrzewa        1085
@laurbuzz            1835
@AnneInVino          3685
@gordone_cellars     4177
@wawinereport        4966
@JoeCz               5147
@mattkettmann        6332
@paulgwine           9532
@vboone              9537
@kerinokeefe        10776
@wineschach         15134
@vossroger          25514
Name: taster_twitter_handle, dtype: int64

In [ ]:
#Q2.What is the best wine I can buy for a given amount of money? 
#Create a Series whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. 
#Sort the values by price, ascending 
best_rating_per_price =reviews.groupby('price').points.max()
best_rating_per_price.head(5)

price
4.0    86
5.0    87
6.0    88
7.0    91
8.0    91
Name: points, dtype: int64

In [ ]:
#Q3.Create a DataFrame whose index is the variety category from the dataset and whose values are the min and max values thereoF.

# agg(), which lets you run a bunch of different functions on your DataFrame simultaneously
price_extremes = reviews.groupby('variety').price.agg([min, max])
price_extremes.head(5)

,min,max
variety,,
Abouriou,15.0,75.0
Agiorgitiko,10.0,66.0
Aglianico,6.0,180.0
Aidani,27.0,27.0
Airen,8.0,10.0


In [ ]:
#Q4. Create a variable sorted_varieties containing a copy of the dataframe from the previous question where varieties are sorted 
#in descending order based on minimum price, then on maximum price (to break ties).What are the most expensive wine varieties?
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)
sorted_varieties.head(5)

,min,max
variety,,
Ramisco,495.0,495.0
Terrantez,236.0,236.0
Francisa,160.0,160.0
Rosenmuskateller,150.0,150.0
Tinta Negra Mole,112.0,112.0


In [ ]:
#Q5.Create a Series whose index is reviewers and whose values is the average review score given out by that reviewer. 
#Hint: you will need the taster_name and points columns, and mean()
reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()
reviewer_mean_ratings.head(5)
#reviewer_mean_ratings.describe()

taster_name
Alexander Peartree    85.855422
Anna Lee C. Iijima    88.415629
Anne Krebiehl MW      90.562551
Carrie Dykes          86.395683
Christina Pickard     87.833333
Name: points, dtype: float64

In [ ]:
#Q6.What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs.
#Sort the values in the Series in descending order based on wine count.
#Hint: Use groupby(), and provide a list of columns to group by. You may also find size() and sort_values() to be useful!
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)
country_variety_counts.head(5)
#The size() function is used to get an int representing the number of elements in this object.
#Return the number of rows if Series. Otherwise return the number of rows times number of columns if DataFrame.

country  variety                 
US       Pinot Noir                  9885
         Cabernet Sauvignon          7315
         Chardonnay                  6801
France   Bordeaux-style Red Blend    4725
Italy    Red Blend                   3624
dtype: int64

**Exercise: Data Types and Missing Values**

In [ ]:
#Q1. data type of the points column in the dataset
#datatype of dataset
reviews.describe
reviews.points.dtype #data type for a column in a DataFrame or a Series is known as the dtype.

dtype('int64')

In [ ]:
#Q2.Series from entries in the points column, but convert the entries to strings
point_strings=reviews.points.astype(str)
point_strings
#to convert a column of one type into another wherever such a conversion makes sense by using the astype()

#float64 -64-bit floating point number, int64-64 bit sized integer, 
#str-columns consisting entirely of strings do not get their own type; they are instead given the object type

0         87
1         87
2         87
3         87
4         87
          ..
129966    90
129967    90
129968    90
129969    90
129970    90
Name: points, Length: 129971, dtype: object

In [ ]:
#Q3.Sometimes the price column is null. How many reviews in the dataset are missing a price?
missing_prices=reviews[reviews.price.isnull()]
n_missing_prices=len(missing_prices)
n_missing_prices#no. of missing prices in the dataset 

8996

In [ ]:
#Q4. Create a Series counting the number of times each value occurs in the region_1 field. This field is often missing data, so replace missing values with Unknown. Sort in descending order.
missing_values_region_1=reviews[reviews.region_1.isnull()]
#print(missing_values_region_1)
reviews_per_region=reviews.region_1.fillna('Unknown').value_counts().sort_values(ascending=False)
print(reviews_per_region)

Unknown                           21247
Napa Valley                        4480
Columbia Valley (WA)               4124
Russian River Valley               3091
California                         2629
                                  ...  
Paso Robles Highlands District        1
Monterey County-Napa County           1
Napa Valley-Paso Robles               1
Altos de Mendoza                      1
Burgundy                              1
Name: region_1, Length: 1230, dtype: int64


In [ ]:
#Alternatively, we may have a non-null value that we would like to replace. For example, suppose that since this dataset was published, 
#reviewer Kerin O'Keefe has changed her Twitter handle from @kerinokeefe to @kerino. 
#One way to reflect this in the dataset is using the replace() method
rename_handle=reviews.taster_twitter_handle.replace("@kerinokeefe","@kerino").value_counts().sort_values(ascending=True)
rename_handle

@winewchristina         6
@bkfiona               27
@worldwineguys       1005
@suskostrzewa        1085
@laurbuzz            1835
@AnneInVino          3685
@gordone_cellars     4177
@wawinereport        4966
@JoeCz               5147
@mattkettmann        6332
@paulgwine           9532
@vboone              9537
@kerino             10776
@wineschach         15134
@vossroger          25514
Name: taster_twitter_handle, dtype: int64

**Exercise: Renaming and Combining**

In [ ]:
#Q1.rename columns region_1 and region_2 to region and locale, respectively
renamed=reviews.rename(columns={'region_1':'region', 'region_2':'locale'})
renamed.head(2)

,country,description,designation,points,price,province,region,locale,taster_name,taster_twitter_handle,title,variety,winery,points_to_price_ratio,star_ratings
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,NaN,2
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,5.8,2


In [ ]:
#Q2. Set the index name in the dataset to wines
reindexed = reviews.rename_axis('wine nos.', axis='columns')#renaming the index column as 'wines'
reindexed.head(1)

wine nos.,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,points_to_price_ratio,star_ratings
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,NaN,2


In [ ]:
#Pandas has three core methods for doing this. In order of increasing complexity, these are concat(), join(), and merge(). 
#Most of what merge() can do can also be done more simply with join()

# combining 2 datasets
#combined_=pd.concat([name of dataset 1, name of dataset 2]) #datasets are combined without any  

#example
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
print(pd.concat([s1, s2]))
#ignore index
print(pd.concat([s1, s2], ignore_index=True))

0    a
1    b
0    c
1    d
dtype: object
0    a
1    b
2    c
3    d
dtype: object


In [ ]:
#join() lets you combine different DataFrame objects which have an index in common.
#The lsuffix and rsuffix parameters are necessary here because the data has the same column names in both dstasets.

#example
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})
print(df1)
print(df2)
df3=df1.set_index('value').join(df2.set_index('value'))
print(df3)

#merging
df4=df1.merge(df2, left_on='lkey', right_on='rkey')
df4=df4.rename(columns={'value_x':'value_ikey', 'value_y':'value_rkey'}).rename_axis('index_no', axis='columns')
print(df4)

#refer more from Halliburton merge/join

  lkey  value
0  foo      1
1  bar      2
2  baz      3
3  foo      5
  rkey  value
0  foo      5
1  bar      6
2  baz      7
3  foo      8
      lkey rkey
value          
1      foo  NaN
2      bar  NaN
3      baz  NaN
5      foo  foo
index_no lkey  value_ikey rkey  value_rkey
0         foo           1  foo           5
1         foo           1  foo           8
2         foo           5  foo           5
3         foo           5  foo           8
4         bar           2  bar           6
5         baz           3  baz           7
